In [ ]:
# Heavily inspired by / adapted from the MIT RACECAR Course
# FIRST import all the necessary libraries and modules!
import cv2               # import OpenCV
import numpy as np       # import NumPy
import sys
from google.colab import files
from google.colab.patches import cv2_imshow # colab requires its own version of imshow

# Contours

<p style='font-size:1.75rem;line-height:1.5'>
In this lab, we will learn about contours and how to use contours to help us identify objects. Furthermore, we will be learning about the following functions: 
    <ul style='font-size:1.75rem;line-height:1.5'>
        <li> <code> cv2.threshold </code>  </li>
        <li> <code> cv2.findContours </code>  </li>
        <li> <code> cv2.drawContours </code>  </li>
        <li> <code> cv2.contourArea </code>  </li>
        <li> <code> cv2.boundingRect </code>  </li>
        <li> <code> cv2.minAreaRect </code>  </li>
        <li> <code> cv2.minEnclosingCircle</code>  </li>
    </ul>
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'>  
    We will learn how to create a function to help us detect drawings on a piece of paper! 
    </p>

### What is a contour? 

<p style='font-size:1.75rem;line-height:1.5'>
    A contour is a curve joining all continuous points along a boundary.
    </p>


### Why contours?

<p style='font-size:1.75rem;line-height:1.5'>
    Contours can help us analyze an object's shape and detect objects, so they will be helpful in the future when we want detect specific objects or obstacles in the car's field of view (ie. cones).
    </p>

<img src="http://4.bp.blogspot.com/-ZNhledLRAyo/ULYPHPSd8rI/AAAAAAAAANk/WwRi4jhW7cU/s1600/result.png" alt="object contour detection" style="width: 300px;"/>

---

# Find and Draw Contours

<p style='font-size:1.75rem;line-height:1.5'>
    In the following section, we will learn how to find and draw contours. 
    <br>To find the contours of an image, a few steps need to be taken: 
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    <b style="color:blue">STEP #1:</b> Read the image.
    </p>

In [ ]:
# TASK #1: Read 'star.png' via cv2.imread. Save as 'img'.
uploaded = files.upload()
img = cv2.imread("star.png", cv2.IMREAD_COLOR)

In [ ]:
# Save the original image to display later.
img_copy = img.copy()

# Show image
cv2_imshow(img)

<p style='font-size:1.75rem;line-height:1.5'>
    <b style="color:blue">STEP #2:</b> Convert the image into grayscale.
    </p>

In [ ]:
# TASK #2: Convert the BGR image to grayscale via cv2.COLOR_BGR2GRAY. 
#          Save as 'img_gray' 
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Show image in popup window
cv2_imshow(img_gray)

<p style='font-size:1.75rem;line-height:1.5'>
    <b style="color:blue">STEP #3:</b> Convert the grayscale image into a binary image with <code>cv2.threshold</code>. 
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    <code>cv2.threshold</code> looks at each pixel to see if it is greater than or less than the threshold value, and then reassigns the input minVal/maxVal to the pixel.
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    It has the following format: 
    </p>
    
```python
thresh = cv2.threshold(<grayscale_image>, <threshold_value>, <maxVal>, <minVal>)[1]
```

In [ ]:
# TASK #3: Threshold the image. Save as 'thresh'.
#          Use threshold_value=240, maxVal=255, and minVal=0
thresh = cv2.threshold(img_gray, 240, 255, 0)[1]

# Show image in popup window
cv2_imshow(thresh)

<p style='font-size:1.75rem;line-height:1.5'>
    <b style="color:blue">STEP #4:</b> Invert the threshold mask
    </p>

In [ ]:
# TASK #4: Invert the threshold mask via cv2.bitwise_not. Save as 'mask'
mask = cv2.bitwise_not(thresh)

# Show image in popup window
cv2_imshow(mask)

<p style='font-size:1.75rem;line-height:1.5'>
    <b style="color:blue">STEP #5:</b> Use <code>cv2.findContours</code> to find the list of contours.
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    It has the following format:
    </p>
    
```python
contours = cv2.findContours(<mask>, 3, 2)[0]
```

In [ ]:
# TASK #5: Find contours of 'mask' via cv2.findContours. Save as 'contours'.
contours = cv2.findContours(mask, 3, 2)[0]

# Print contours found note that for this image there should only be one
# object so the length is 1 and then there are a series of points that
# define its boundary
print(len(contours))
print(contours)

<p style='font-size:1.75rem;line-height:1.5'>
    <b style="color:blue">STEP #6:</b> Draw contours via <code>cv2.drawContours</code>.
    </p>

<p style='font-size:1.75rem;line-height:1.5'>
    It has the following format:
    </p>
    
```python
cv2.drawContours(<image>, <contours>, <contour_index>, <color>, <thickness>)
```

<p style='font-size:1.75rem;line-height:1.5'>
    <b>Notes:</b> 
    <br> Use <code>-1</code> for <code>&lt;contour_index&gt;</code> to draw all contours. 
    <br> To draw a specific contour, index into the list via <code>[contours[i]]</code> for <code>&lt;contours&gt;</code>, and set <code>0</code> for <code>&lt;contour_index&gt;</code>
    </p>

In [ ]:
# TASK #6: Draw GREEN contours via cv2.drawContours. 
#          Use contour_index=-1 and thickness=3.
cv2.drawContours(img, contours, -1, (0,255,0), 3)

# Show image in popup window
cv2_imshow(img)

# Contour Features and Properties
<p style='font-size:1.75rem;line-height:1.5'>
    Now that we know a bit more about contours and how to use them, lets extract some data from them! 
    </p>

### Contour Area
<p style='font-size:1.75rem;line-height:1.5'>
    <code>cv2.contourArea(&lt;single_contour&gt;)</code> calculates the total area that a contour encloses. 
</p> 

<p style='font-size:1.75rem;line-height:1.5'>
    It is necessary to select a <b>single contour</b> when looking for the area, even when <code>contours</code> only has one contour stored in it.
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'>
    <br> Lets find the contour area of <code>star.png</code>
    </p>

In [ ]:
# Get contours of star.png
img = cv2.imread('star.png')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(img_gray, 240, 255, 0)[1]
inverted_thresh = cv2.bitwise_not(thresh)
contours = cv2.findContours(inverted_thresh, 3, 2)[0]


# TASK #1: Index into the first contour and save it to 'cnt'.
cnt = contours[0]

# TASK #2: Get the contour area of the first contour
area = cv2.contourArea(cnt)

# TASK #3: Print the area
print(area)

# Show the first contour 
cv2.drawContours(img, [cnt], 0, (255, 0, 0), 3)
cv2_imshow(img)

### Straight Bounding Rectangle

<p style='font-size:1.75rem;line-height:1.5'>
    We can use <code>cv2.boundingRect</code> to draw a rectangle. 
    <br> It has the following format: 
    </p>
    
 ```python
x, y, w, h = cv2.boundingRect(<single_contour>)
 ```

<p style='font-size:1.75rem;line-height:1.5'>
    <br> Lets find the straight bounding rectangle of <code>bolt.jpg</code>
    </p>

In [ ]:
# Upload bolt.jpg
uploaded = files.upload()

In [ ]:
# Get contours of bolt.jpg
img = cv2.imread("bolt.jpg", cv2.IMREAD_COLOR)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(img_gray, 240, 255, 0)[1]
contours = cv2.findContours(thresh, 3, 2)[0]

# TASK #1: Get bounding rectangle of the first contour
x, y, w, h = cv2.boundingRect(contours[0])

# TASK #2: Draw a GREEN bounding rectangle using cv2.rectangle
img = cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 3)

# Show bounding rectangle in popup window
cv2_imshow(img)

### Minimum Enclosing Rotated Box

<p style='font-size:1.75rem;line-height:1.5'>
    Another bounding rectangle is <code>cv2.minAreaRect</code>. This rectangle encloses the contour with the smallest area, and can be rotated.
    </p>
    
<p style='font-size:1.75rem;line-height:1.5'>
     The function has the following format:
     </p>
     
```python
rect = cv2.minAreaRect(<single_contour>)
```

<img src="https://docs.opencv.org/3.1.0/boundingrect.png" width="150" height="150"> </img>

<p style='font-size:1.75rem;line-height:1.5'>
    <b style="color:red">Exercise:</b> 
    <br> Find the minimum enclosing rectangle of <code>bolt.jpg</code>
    </p>

In [ ]:
# Get contours of bolt.png
img = cv2.imread('bolt.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(img_gray, 240, 255, 0)[1]
contours = cv2.findContours(thresh, 3, 2)[0]


# TASK: Get min area bounding rectangle of the first contour. 
#       Save as 'rect'.
rect = cv2.minAreaRect(contours[0])

# Draw the rectangle
box = np.int0(cv2.boxPoints(rect))
res = cv2.drawContours(img, [box], 0, (0,0,255), 2)

# Show min area bounded rectangle
cv2_imshow(res)

<p style='font-size:1.75rem;line-height:1.5'>
    <b>Optional Note on <code>cv2.minAreaRect</code>:</b>
    </p>
<code>rect = cv2.minAreaRect(&lt;contour&gt;)</code> will output rectangle data so that <code>box_points = cv2.boxPoints(rect)</code> can calculate box points. The box points will be floating data types at first, so we want to use <code>np.int0(box_points)</code> to convert the datatype from floats to integers. Then, we can use <code>cv2.drawContours</code> to draw the minimum enclosed (rotated) box around the image.

### Minimum Enclosing Circle

<p style='font-size:1.75rem;line-height:1.5'>
    <code>cv2.minEnclosingCircle</code> gives us the center and radius of the minimum enclosing circle of a contour.
    </p>
        
<p style='font-size:1.75rem;line-height:1.5'>
     The function has the following format:
     </p>
     
```python
(x,y), radius = cv2.minEnclosingCircle(<single_contour>)
```

<p style='font-size:1.75rem;line-height:1.5'>
    <b style="color:red">Exercise:</b> 
    <br> Find the minimum enclosing circle of <code>bolt.jpg</code>
    </p>

In [ ]:
# Get contours of bolt.png
img = cv2.imread('bolt.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(img_gray, 240, 255, 0)[1]
contours = cv2.findContours(thresh, 3, 2)[0]

# TASK #1: Get the minimum enclosing circle of the first contour
(x,y), radius = cv2.minEnclosingCircle(contours[0])

# Convert to values to int
center = (int(x),int(y))
radius = int(radius)

# TASK #2: Draw a GREEN bounding circle using cv2.circle
img = cv2.circle(img, center, radius, (0,255,0), 3)

# Show image in popup
cv2_imshow(img)


---


# Challenge Draw it All -- On a Stop Sign!

<p style='font-size:1.75rem;line-height:1.5'>
    Let's put everything we've learned altogether now! Draw a straight bounded rectangle, a minimum area rectangle, and a minimum enclosing circle around an image of a stop sign that you find on the internet. Make sure to use different colors for the different contours! Be careful! You will likely find multiple contours! Make sure you find the right one! Hint: you will also likely need to mask on a color image vs. just thresholding! There are lots of ways to do that (e.g., cv2.inRange)! Google around!
</p> 

In [ ]:
# TASK #1: Read the image 'TBD'. Save as 'img'


# TASK #2: Convert the image into grayscale via cv2.cvtcolor


# TASK #3: Threshold the image via cv2.threshold.
#          You will likely need to play around with the threshold values!


# TASK #4: Find the list of contours of 'thresh' via cv2.findContours


# TASK #5: Use cv2.boundingRect to find x, y, w, h


# TASK #6: Draw a BLUE straight rectangle 


# TASK #7: Use cv2.minAreaRect to find the minimum enclosing rectangle. 
#          Save as 'rect'


# Draw minimum enclosing rectangle
box = np.int0(cv2.boxPoints(rect))
res = cv2.drawContours(img, [box], 0, (0,255,0), 2)

# TASK #8: Use cv2.minEnclosingCircle to find radius and (x,y)


# Convert values to int
(x,y) = int(x), int(y)
radius = int(radius)

# TASK #9: Draw the RED circle 


# Show the image
cv2_imshow(img)


---
